<a href="https://colab.research.google.com/github/jeraldkim/DataScienceProjects/blob/main/Scraping_Anime_from_myanimelist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re
import time
import numpy as np
from google.colab import drive
from google.colab.data_table import DataTable

In [7]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# path = "drive/MyDrive/24000anime021823.csv"
# anime_df = pd.read_csv(path).drop(columns=["Unnamed: 0"])

In [4]:
#@title Use this to Scrape Anime Links
anime_list = []
num_anime = 480 #ex: 5 means 5*50 we want top 250 anime 0 would mean only top 50 anime so not run loop 200 = top 10000 anime
n = 0

while n <= num_anime:
    if n == 0:
      curr_link = "https://myanimelist.net/topanime.php"
    elif n > 0:
      curr_link = "https://myanimelist.net/topanime.php?limit=" + str(n * 50)
    n += 1

    response = requests.get(curr_link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")
    table = soup.find("table", class_="top-ranking-table")
    time.sleep(3) #necessary condition to prevent site from blocking your requests


    if table == None:
      print(curr_link)
      print(soup)
      bug1 = response
      bug2 = html_content
      bug3 = soup
    
    for row in table.find_all("tr")[1:51]:
      anime = {}
      rank = row.find("td", class_="rank").text.strip()
      if rank == '-':
        anime["Rank"] = None
      else:
        anime["Rank"] = int(rank)
      anime["Title"] = row.find("div", class_="di-ib clearfix").find("a").text
      anime["Link"] = row.find("a").get("href")
      score = row.find("td", class_="score").text.strip()
      if score == 'N/A':
        anime["Score"] = None
      else:
        anime["Score"] = float(score)
      anime_list.append(anime)

anime_df = pd.DataFrame(anime_list)

In [5]:
anime_df

,Rank,Title,Link,Score
0,None,Doukyuusei (OVA): Climax,https://myanimelist.net/anime/4841/Doukyuusei_...,5.33
1,None,Flashback Game,https://myanimelist.net/anime/3705/Flashback_Game,5.33
2,None,Pigeon Blood,https://myanimelist.net/anime/2351/Pigeon_Blood,5.33
3,None,Rankou Choukyou: Maid ni Natta Shoujo,https://myanimelist.net/anime/3094/Rankou_Chou...,5.33
4,None,S-ke ni Totsuida M-jou no Nichijou,https://myanimelist.net/anime/48467/S-ke_ni_To...,5.33
...,...,...,...,...
248,None,Ikumonogakari The Animation,https://myanimelist.net/anime/53678/Ikumonogak...,NaN
249,None,Isaku: Tsumi to Batsu,https://myanimelist.net/anime/43015/Isaku__Tsu...,NaN
250,None,Ke Ai,https://myanimelist.net/anime/52630/Ke_Ai,NaN
251,None,Tsumamigui The Animation Chance.0: Watashi wo ...,https://myanimelist.net/anime/51763/Tsumamigui...,NaN


In [ ]:
drive.mount('/content/drive')
path = '/content/drive/My Drive/24000anime021823.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  anime_df.to_csv(f)

#method to save df to csv file on google drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# real = anime_df["Link"] #for generating entire data

# tests = anime_scraped_df["Link"][:5] #use this one to test for implementation
new_anime_list = []
#downloaded = anime_scraped_df

curr = 22001 #change these for where you stopped off
n = 23000

while n <= 24000:
  new_anime_list = []

  for link in anime_df[curr-1:n]["Link"]: #changed curr-1 to include 0 index

# for link in tests: #change here for testing

      response = requests.get(link)
      html_content = response.content
      soup = BeautifulSoup(html_content, "html.parser")
      time.sleep(3)

      anime = {}
      anime_type = soup.find("span",class_ ="dark_text",string="Type:")
      if anime_type == None:
        anime["Type"] = None
      else:
        anime["Type"] = anime_type.find_parent().text.replace('Type:','').strip() #Type of Show
      num_episodes = soup.find("span",class_ ="dark_text",string="Episodes:")
      if num_episodes == None:
        anime["# Episodes"] = None
      else:
        anime["# Episodes"] = num_episodes.next_sibling.strip() #Number of Episodes
    
      source = soup.find("span",class_ ="dark_text",string="Source:")
      if source == None:
        anime["Source"] = None
      else:
        anime["Source"] = source.next_sibling.strip() #Source

      status = soup.find("span",class_ ="dark_text",string="Status:")
      if status == None:
        anime["Status"] = None
      else:
        anime["Status"] = status.next_sibling.strip()

      has_premiered = soup.find("span",class_ ="dark_text",string="Premiered:")
      if has_premiered == None:
        anime["Premiered"] = None
      else:
        anime["Premiered"] = re.sub('Premiered:','',has_premiered.find_parent().text).strip() #Season of Anime and Year Fix for Premiered: ? Case
    
      aired_date = soup.find("span",class_ ="dark_text",string="Aired:")
      if aired_date == None:
        anime["Aired Date"] = None
      else:
        anime["Aired Date"] = aired_date.next_sibling.strip() #Aired Date
    
      studio = soup.find("span",class_ ="dark_text",string="Studios:")
      if studio == None:
        anime["Studios"] = None
      else: 
        anime["Studios"] = re.sub('\nStudios:\n','',studio.find_parent().text).strip() #Studio RECENTLy made a regex change for animes with more than 1 studio (rare though)

      has_genre = soup.find("span",class_ ="dark_text",string=re.compile(r'Genres:|Genre:')) #FIXED to scrape multiple genres
      if has_genre == None:
        anime["Genres"] = None
      else:
        genres = has_genre.find_parent().find_all("a") #Genre
        genre_list = ""
        for genre in genres: #add as a string
          genre_list += genre.text.strip() + ","
        anime["Genres"] = genre_list
      
      has_theme = soup.find("span",class_ ="dark_text",string=re.compile(r'Themes:|Theme:')) #NEW addition checks for Themes AND Theme
      if has_theme == None:
        anime["Themes"] = None
      else:
        themes = has_theme.find_parent().find_all("a")
        themes_list = ""
        for theme in themes:
          themes_list += theme.text.strip() + ","
        anime["Themes"] = themes_list


      demo = soup.find("span",class_ ="dark_text",string="Demographic:")
      if demo == None:
        anime["Demographic"] = None
      else:
        anime["Demographic"] = demo.find_parent().find("a").text #Demographic
    
      duration = soup.find("span",class_ ="dark_text",string="Duration:")
      if duration == None:
        anime["Duration"] = None
      else:
        anime["Duration"] = duration.next_sibling.strip() #Duration

      age_rating = soup.find("span",class_ ="dark_text",string="Rating:")
      if age_rating == None:
        anime["Age Rating"] = None
      else:
        anime["Age Rating"] = age_rating.next_sibling.strip() #Age Rating

      #New feature additions started from this line
      has_scores = soup.find("span",class_ ="dark_text",string="Score:")
      if has_scores == None:
        anime["Number of Raters"] = None
      else:
        has_numraters = has_scores.find_parent().find(itemprop="ratingCount")
        if has_numraters == None:
          anime["Number of Raters"] = None
        else:
          anime["Number of Raters"] = has_numraters.text #Number of people who rated the scores

      popularity = soup.find("span",class_ ="dark_text",string="Popularity:")
      if popularity == None:
        anime["Popularity"] = None
      else:
        anime["Popularity"] = popularity.next_sibling.strip() #Popularity Rank determined by number of "members"

      members = soup.find("span",class_ ="dark_text",string="Members:")
      if members == None:
        anime["Members"] = None
      else:
        anime["Members"] = members.next_sibling.strip() # Number of members = num of  people who added the series to the list


      favorites = soup.find("span",class_ ="dark_text",string="Favorites:")
      if favorites == None:
        anime["Favorites"] = None
      else:
        anime["Favorites"] = favorites.next_sibling.strip() #Favorites

      #Adapation useful for comparing manga scores
      #Sequel
      #Prequel
      has_related = soup.find("table",class_ ="anime_detail_related_anime")
      if has_related == None:
        anime["Adaptation"] = None
        anime["Sequel"] = None
        anime["Prequel"] = None
      else:
        has_adaptation = has_related.find(string="Adaptation:")
        if has_adaptation == None:
          anime["Adaptation"] = None
        else:
          anime["Adaptation"] = has_adaptation.find_parent().next_sibling.text

        has_sequel = has_related.find(string="Sequel:")
        if has_sequel == None:
          anime["Sequel"] = None
        else:
          anime["Sequel"] = has_sequel.find_parent().next_sibling.text

        has_prequel = has_related.find(string="Prequel:")
        if has_prequel == None:
          anime["Prequel"] = None
        else:
          anime["Prequel"] = has_prequel.find_parent().next_sibling.text

      #new addition of voice actor, character, role

      Character = []
      Role = []
      Voice_Actors = []

      char_role_voice = soup.find("div",class_="detail-characters-list clearfix")

      if char_role_voice == None:
        anime["Characters"] = None
        anime["Role"] = None
        anime["Voice Actors"] = None
      else:    
        for child in char_role_voice.children:
        
          characters1 = child.find_all("h3")
          if characters1 == None:
            anime["Characters"] = None
          else:
            for chars in characters1:
              Character += [chars.text]
        
          roles1 = child.find_all("div",class_ = "spaceit_pad")
          if roles1 == None:
            anime["Role"] = None
          else:
            for chars in roles1:
              Role += [chars.text.strip()]

          va1 = child.find_all("td",class_ = "va-t ar pl4 pr4")
          if va1 == None:
            anime["Voice Actors"] = None
          else:
            for chars in va1:
              Voice_Actors += [re.sub('\\n.*','',chars.text.strip())]

      anime["Characters"] = Character
      anime["Role"] = Role
      anime["Voice Actors"] = Voice_Actors   
    
      #addition of music in anime
      opening_songs = []
      opening_artists = []
      ending_songs = []
      ending_artists = []

      has_openings = soup.find("div",class_="theme-songs js-theme-songs opnening")
      if has_openings == None:
        anime["Openings"] = None
        anime["Opening Artists"] = None
      else:
        # openings1 = has_openings.find_all("span",class_="theme-song-title")
        opening_songs = re.findall("\"(.+)\"\sby",has_openings.text)
        # for song in openings1:
        #   opening_songs += [re.sub(r'\'^\'|\'$|^\"|\"$',"",song.text)]

        opening_artists1 = has_openings.find_all("span",class_="theme-song-artist")
        for artist in opening_artists1:
          opening_artists += [re.sub("\sby\s","",artist.text)]
    
        anime["Openings"] = opening_songs
        anime["Opening Artists"] = opening_artists

      has_endings = soup.find("div",class_="theme-songs js-theme-songs ending")
      if has_endings == None:
        anime["Endings"] = None
        anime["Ending Artists"] = None
      else:
        # endings1 = has_endings.find_all("span",class_="theme-song-title")
        ending_songs = re.findall("\"(.+)\"\sby",has_endings.text)
        # for song in endings1:
        #   ending_songs += [re.sub(r'\'^\'|\'$|^\"|\"$',"",song.text)]

        ending_artists1 = has_endings.find_all("span",class_="theme-song-artist")
        for artist in ending_artists1:
          ending_artists += [re.sub("\sby\s","",artist.text)]

        anime["Endings"] = ending_songs
        anime["Ending Artists"] = ending_artists
    
      #Final Step
      new_anime_list.append(anime)    

  new_anime_df = pd.DataFrame(new_anime_list)

  

  path = '/content/drive/My Drive/' + str(curr) + "to" + str(n) + 'anime_stats_021723.csv'

  with open(path, 'w', encoding = 'utf-8-sig') as f:
    new_anime_df.to_csv(f)
  print("uploaded" + str(curr) + "to" + str(n))

  n += 1000
  curr += 1000

uploaded22001to23000
uploaded23001to24000


In [ ]:
# new_anime_df = pd.DataFrame(new_anime_list)
new_anime_df.max_columns = 30
new_anime_df

,Type,# Episodes,Source,Status,Premiered,Aired Date,Studios,Genres,Themes,Demographic,...,Adaptation,Sequel,prequel,Characters,Role,Voice Actors,Openings,Opening Artists,Endings,Ending Artists
0,TV,64,Manga,Finished Airing,Spring 2009,"Apr 5, 2009 to Jul 4, 2010",Bones,"Action,Adventure,Drama,Fantasy,","Military,",Shounen,...,Fullmetal Alchemist,None,None,"[Elric, Edward, Elric, Alphonse, Mustang, Roy,...","[Main, Main, Supporting, Supporting, Supportin...","[Park, Romi, Kugimiya, Rie, Miki, Shinichiro, ...","[again, Hologram (ホログラム), Golden Time Lover (ゴ...","[YUI, NICO Touches the Walls, Sukima Switch, C...","[Uso (嘘), LET IT OUT, Tsunaida Te (つないだ手), Shu...","[SID, Miho Fukuhara, Lil'B, SCANDAL, Nakagawa ..."
1,TV,13,Manga,Finished Airing,Fall 2022,"Oct 11, 2022 to Dec 27, 2022",Pierrot,"Action,Adventure,Fantasy,",None,Shounen,...,Bleach,Bleach: Sennen Kessen-hen - Ketsubetsu-tan,Bleach,"[Kurosaki, Ichigo, Kuchiki, Rukia, Inoue, Orih...","[Main, Main, Main, Main, Main, Main, Supportin...","[Morita, Masakazu, Orikasa, Fumiko, Matsuoka, ...",[スカー],[キタニタツヤ],"[Rapport, 最果て, Number One]","[キタニタツヤ, SennaRin, Shiro Sagisu, Hazel Fernandes]"
2,TV,24,Visual novel,Finished Airing,Spring 2011,"Apr 6, 2011 to Sep 14, 2011",White Fox,"Drama,Sci-Fi,Suspense,","Psychological,Time Travel,",None,...,Steins;Gate,Steins;Gate: Oukoubakko no Poriomania,None,"[Okabe, Rintarou, Makise, Kurisu, Shiina, Mayu...","[Main, Main, Main, Main, Supporting, Supportin...","[Miyano, Mamoru, Imai, Asami, Hanazawa, Kana, ...",[Hacking to the Gate],[Kanako Itou],"[Toki Tsukasadoru Juuni no Meiyaku (刻司ル十二ノ盟約),...","[Yui Sakakibara, Takeshi Abo, Kanako Itou, Kan..."
3,TV,51,Manga,Finished Airing,Spring 2015,"Apr 8, 2015 to Mar 30, 2016",Bandai Namco Pictures,"Action,Comedy,Sci-Fi,","Gag Humor,Historical,Parody,Samurai,",Shounen,...,Gintama,Gintama.,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Ei...,"[Sakata, Gintoki, Kagura, Shimura, Shinpachi, ...","[Main, Main, Main, Supporting, Supporting, Sup...","[Sugita, Tomokazu, Kugimiya, Rie, Sakaguchi, D...","[DAY×DAY, Pride Kakumei (プライド革命), Saigo made I...","[BLUE ENCOUNT, CHiCO with HoneyWorks, Aqua Tim...","[DESTINY, Saigo made II (最後までⅡ), Pride Kakumei...","[Negoto, Aqua Timez, CHiCO with HoneyWorks, TH..."
4,TV,13,Manga,Finished Airing,Spring 2022,"Apr 9, 2022 to Jun 25, 2022",A-1 Pictures,"Comedy,Romance,","School,",Seinen,...,Kaguya-sama wa Kokurasetai: Tensai-tachi no Re...,Kaguya-sama wa Kokurasetai: First Kiss wa Owar...,Kaguya-sama wa Kokurasetai? Tensai-tachi no Re...,"[Shinomiya, Kaguya, Ishigami, Yuu, Fujiwara, C...","[Main, Main, Main, Main, Main, Supporting, Sup...","[Koga, Aoi, Suzuki, Ryouta, Kohara, Konomi, Fu...",[GIRI GIRI feat Suu (GIRI GIRI feat. すぅ)],[Masayuki Suzuki],"[GIRI GIRI feat Suu (GIRI GIRI feat. すぅ), Hear...","[Masayuki Suzuki, Airi Suzuki, Miyuki Shirogan..."


In [ ]:
#run this after scraping all anime
n = 1000
anime_stats_df_list = []
while n <= 24000:
  anime_stats_path = '/content/drive/My Drive/' + str(n-999) + "to" + str(n) + 'anime_stats_021723.csv'
  anime_stats_df_list.append(pd.read_csv(anime_stats_path).drop(columns=["Unnamed: 0"]))
  n += 1000

anime_stats_df_combined = pd.concat(anime_stats_df_list, axis=0, ignore_index=True)

In [ ]:
anime_stats_df_combined

,Type,# Episodes,Source,Status,Premiered,Aired Date,Studios,Genres,Themes,Demographic,...,Adaptation,Sequel,Prequel,Characters,Role,Voice Actors,Openings,Opening Artists,Endings,Ending Artists
0,TV,64,Manga,Finished Airing,Spring 2009,"Apr 5, 2009 to Jul 4, 2010",Bones,"Action,Adventure,Drama,Fantasy,","Military,",Shounen,...,Fullmetal Alchemist,NaN,NaN,"['Elric, Edward', 'Elric, Alphonse', 'Mustang,...","['Main', 'Main', 'Supporting', 'Supporting', '...","['Park, Romi', 'Kugimiya, Rie', 'Miki, Shinich...","['again', 'Hologram (ホログラム)', 'Golden Time Lov...","['YUI', 'NICO Touches the Walls', 'Sukima Swit...","['Uso (嘘)', 'LET IT OUT', 'Tsunaida Te (つないだ手)...","['SID', 'Miho Fukuhara', ""Lil'B"", 'SCANDAL', '..."
1,TV,13,Manga,Finished Airing,Fall 2022,"Oct 11, 2022 to Dec 27, 2022",Pierrot,"Action,Adventure,Fantasy,",NaN,Shounen,...,Bleach,Bleach: Sennen Kessen-hen - Ketsubetsu-tan,Bleach,"['Kurosaki, Ichigo', 'Kuchiki, Rukia', 'Inoue,...","['Main', 'Main', 'Main', 'Main', 'Main', 'Main...","['Morita, Masakazu', 'Orikasa, Fumiko', 'Matsu...",['スカー'],['キタニタツヤ'],"['Rapport', '最果て', 'Number One']","['キタニタツヤ', 'SennaRin', 'Shiro Sagisu, Hazel Fe..."
2,TV,24,Visual novel,Finished Airing,Spring 2011,"Apr 6, 2011 to Sep 14, 2011",White Fox,"Drama,Sci-Fi,Suspense,","Psychological,Time Travel,",NaN,...,Steins;Gate,Steins;Gate: Oukoubakko no Poriomania,NaN,"['Okabe, Rintarou', 'Makise, Kurisu', 'Shiina,...","['Main', 'Main', 'Main', 'Main', 'Supporting',...","['Miyano, Mamoru', 'Imai, Asami', 'Hanazawa, K...",['Hacking to the Gate'],['Kanako Itou'],['Toki Tsukasadoru Juuni no Meiyaku (刻司ル十二ノ盟約)...,"['Yui Sakakibara', 'Takeshi Abo', 'Kanako Itou..."
3,TV,51,Manga,Finished Airing,Spring 2015,"Apr 8, 2015 to Mar 30, 2016",Bandai Namco Pictures,"Action,Comedy,Sci-Fi,","Gag Humor,Historical,Parody,Samurai,",Shounen,...,Gintama,Gintama.,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Ei...,"['Sakata, Gintoki', 'Kagura', 'Shimura, Shinpa...","['Main', 'Main', 'Main', 'Supporting', 'Suppor...","['Sugita, Tomokazu', 'Kugimiya, Rie', 'Sakaguc...","['DAY×DAY', 'Pride Kakumei (プライド革命)', 'Saigo m...","['BLUE ENCOUNT', 'CHiCO with HoneyWorks', 'Aqu...","['DESTINY', 'Saigo made II (最後までⅡ)', 'Pride Ka...","['Negoto', 'Aqua Timez', 'CHiCO with HoneyWork..."
4,TV,13,Manga,Finished Airing,Spring 2022,"Apr 9, 2022 to Jun 25, 2022",A-1 Pictures,"Comedy,Romance,","School,",Seinen,...,Kaguya-sama wa Kokurasetai: Tensai-tachi no Re...,Kaguya-sama wa Kokurasetai: First Kiss wa Owar...,Kaguya-sama wa Kokurasetai? Tensai-tachi no Re...,"['Shinomiya, Kaguya', 'Ishigami, Yuu', 'Fujiwa...","['Main', 'Main', 'Main', 'Main', 'Main', 'Supp...","['Koga, Aoi', 'Suzuki, Ryouta', 'Kohara, Konom...",['GIRI GIRI feat Suu (GIRI GIRI feat. すぅ)'],['Masayuki Suzuki'],"['GIRI GIRI feat Suu (GIRI GIRI feat. すぅ)', 'H...","['Masayuki Suzuki', 'Airi Suzuki', 'Miyuki Shi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,OVA,2,Unknown,Finished Airing,NaN,"Oct 11, 1984 to Oct 7, 1985","None found, add some","Hentai,",NaN,NaN,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
23996,OVA,1,Manga,Finished Airing,NaN,"Apr 27, 1990",J.C.Staff,"Comedy,Erotica,","Harem,",NaN,...,NaN,Ikenai Boy: Ikasu Maruhi Hand Power,NaN,[],['Animation Director'],[],[],[],[],[]
23997,OVA,2,Unknown,Finished Airing,NaN,"Oct 21, 1994 to Nov 25, 1995",J.C.Staff,"Hentai,",NaN,NaN,...,NaN,NaN,Sei Michaela Gakuen Hyouryuuki,"['Kozue', 'Miuchi, Youko', 'Kumi', 'Gonza', 'S...","['Main', 'Supporting', 'Supporting', 'Supporti...",[],[],[],[],[]
23998,OVA,1,Manga,Finished Airing,NaN,"Feb 25, 1998",Ripple Film,"Hentai,",NaN,NaN,...,NaN,NaN,NaN,['Kaori'],['Main'],[],[],[],[],[]


In [ ]:
final_anime_df =anime_df.join(anime_stats_df_combined)

In [ ]:
final_anime_df[final_anime_df["Studios"] == 'Kyoto Animation'].mean()

<ipython-input-14-54c8c1d2452c>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_anime_df[final_anime_df["Studios"] == 'Kyoto Animation'].mean()


Rank                  3240.070796
Score                    7.313805
Number of Raters    129942.654867
dtype: float64

In [ ]:
final_anime_df.groupby(by = "Type").mean().sort_values("Score",ascending=False)

,Rank,Score,Number of Raters
Type,,,
TV,10262.690659,6.857023,78143.577221
Special,9245.505882,6.399284,8673.353530
Movie,11727.893515,6.382112,21066.325436
OVA,10336.167933,6.254712,7881.816744
Unknown,8719.000000,6.140000,155.000000
ONA,12797.153532,6.083212,5724.158242
Music,12822.557301,5.921534,931.142330


In [ ]:
final_anime_df[final_anime_df["Type"] == "Unknown"]

,Rank,Title,Link,Score,Type,# Episodes,Source,Status,Premiered,Aired Date,...,Adaptation,Sequel,Prequel,Characters,Role,Voice Actors,Openings,Opening Artists,Endings,Ending Artists
8718,8719.0,Sekai Meisaku Douwa,https://myanimelist.net/anime/7398/Sekai_Meisa...,6.14,Unknown,20,Unknown,Finished Airing,NaN,Oct 1975 to Feb 1983,...,NaN,NaN,NaN,[],"['Episode Director, Storyboard', 'Background A...",[],[],[],[],[]
22131,NaN,Kyoukai Senki: Kyokkou no Souki,https://myanimelist.net/anime/54495/Kyoukai_Se...,NaN,Unknown,Unknown,Original,Not yet aired,NaN,Not available,...,NaN,NaN,NaN,[],"['Director', 'Music', 'Mechanical Design', 'Me...",[],[],[],[],[]
22142,NaN,Lycoris Recoil (Shinsaku Animation),https://myanimelist.net/anime/54440/Lycoris_Re...,NaN,Unknown,Unknown,Original,Not yet aired,NaN,Not available,...,NaN,NaN,NaN,[],[],[],[],[],[],[]
22143,NaN,Made in Abyss: Retsujitsu no Ougonkyou (Zoku-hen),https://myanimelist.net/anime/54250/Made_in_Ab...,NaN,Unknown,Unknown,Web manga,Not yet aired,NaN,Not available,...,Made in Abyss,NaN,Made in Abyss: Retsujitsu no Ougonkyou,[],['Original Creator'],[],[],[],[],[]
22144,NaN,Mahou Shoujo Ikusei Keikaku: Restart,https://myanimelist.net/anime/54344/Mahou_Shou...,NaN,Unknown,Unknown,Light novel,Not yet aired,NaN,Not available,...,Mahou Shoujo Ikusei Keikaku,NaN,Mahou Shoujo Ikusei Keikaku,[],"['Director', 'Original Character Design', 'Ori...",[],[],[],[],[]
22147,NaN,Mahouka Koukou no Rettousei (Zoku-hen),https://myanimelist.net/anime/50713/Mahouka_Ko...,NaN,Unknown,Unknown,Light novel,Not yet aired,NaN,Not available,...,Mahouka Koukou no Rettousei,NaN,Mahouka Koukou no Rettousei: Raihousha-hen,"['Kuroba, Ayako']",['Main'],"['Uchida, Maaya']",[],[],[],[]
22151,NaN,Majo to Yajuu,https://myanimelist.net/anime/52816/Majo_to_Yajuu,NaN,Unknown,Unknown,Manga,Not yet aired,NaN,Not available,...,Majo to Yajuu,NaN,NaN,"['Guideau', 'Ashaf']","['Main', 'Main']",[],[],[],[],[]
22153,NaN,Maou no Ore ga Dorei Elf wo Yome ni Shitanda g...,https://myanimelist.net/anime/53434/Maou_no_Or...,NaN,Unknown,Unknown,Light novel,Not yet aired,NaN,Not available,...,Maou no Ore ga Dorei Elf wo Yome ni Shitanda g...,NaN,NaN,"['Nephilia', 'Zagan']","['Main', 'Main']",[],[],[],[],[]
22167,NaN,Monkey Gang Anarchy,https://myanimelist.net/anime/38910/Monkey_Gan...,NaN,Unknown,Unknown,Novel,Not yet aired,NaN,Not available,...,NaN,NaN,NaN,[],"['Director', 'Music']",[],[],[],[],[]
22170,NaN,Murai no Koi,https://myanimelist.net/anime/52080/Murai_no_Koi,NaN,Unknown,Unknown,Manga,Not yet aired,NaN,Not available,...,Murai no Koi,NaN,NaN,[],['Original Creator'],[],[],[],[],[]


In [ ]:
final_anime_df[final_anime_df["Title"].duplicated()]

,Rank,Title,Link,Score,Type,# Episodes,Source,Status,Premiered,Aired Date,...,Adaptation,Sequel,Prequel,Characters,Role,Voice Actors,Openings,Opening Artists,Endings,Ending Artists
3800,3801.0,Aru Tabibito no Nikki,https://myanimelist.net/anime/1205/Aru_Tabibit...,7.06,ONA,6,Original,Finished Airing,NaN,"Mar 19, 2003 to Jun 4, 2003",...,NaN,NaN,NaN,"['Roddle, Tortov']",['Main'],[],[],[],[],[]
3890,3891.0,Gudetama,https://myanimelist.net/anime/23539/Gudetama,7.05,TV,Unknown,Original,Currently Airing,Spring 2014,"Mar 31, 2014 to ?",...,NaN,NaN,NaN,"['Gudetama', 'Nisetama-san']","['Main', 'Supporting']",[],[],[],[],[]
9000,9001.0,Yomigaeru Sora Pilot,https://myanimelist.net/anime/24527/Yomigaeru_...,6.08,Special,1,Original,Finished Airing,NaN,"Oct 27, 2006",...,NaN,NaN,NaN,[],[],[],[],[],[],[]
15050,15051.0,Neko no Nyagh: Nya Misérables,https://myanimelist.net/anime/38857/Neko_no_Ny...,NaN,TV,25,Game,Finished Airing,Spring 2019,"Apr 3, 2019 to Sep 18, 2019",...,NaN,NaN,NaN,"['Teo', 'Niagho']","['Main', 'Main']","['Kayou, Hikaru', 'Sugita, Tomokazu']",[],[],[],[]
16050,16051.0,Sai Er Hao Season 6: Cangqiong Liehuo,https://myanimelist.net/anime/44148/Sai_Er_Hao...,NaN,TV,30,Game,Finished Airing,Winter 2016,"Jan 1, 2016 to Apr 9, 2016",...,NaN,Sai Er Hao Season 7: Yuzhou Zhi Yan,Sai Er Hao Season 5: Lie Tian Kun Shou,[],[],[],[],[],[],[]
16051,16052.0,Sai Er Hao Season 7: Yuzhou Zhi Yan,https://myanimelist.net/anime/44149/Sai_Er_Hao...,NaN,TV,30,Game,Finished Airing,Summer 2016,"Jun 3, 2016 to Sep 17, 2016",...,NaN,Sai Er Hao Season 8: Huanmeng Zhan Ji,Sai Er Hao Season 6: Cangqiong Liehuo,[],[],[],[],[],[],[]
17050,17051.0,Taka no Tsume-dan no Ike! ODAman,https://myanimelist.net/anime/40393/Taka_no_Ts...,NaN,ONA,Unknown,Original,Currently Airing,NaN,"Sep 20, 2018 to ?",...,NaN,NaN,NaN,[],[],[],[],[],[],[]
17051,17052.0,Taka no Tsume-dan x Uchicomi!,https://myanimelist.net/anime/40342/Taka_no_Ts...,NaN,ONA,5,Original,Finished Airing,NaN,"Aug 19, 2019 to Aug 29, 2019",...,NaN,NaN,NaN,[],[],[],[],[],[],[]
17052,17053.0,"Taka no Tsume-dan Yoshida Presents ""Sengoku Ta...",https://myanimelist.net/anime/39762/Taka_no_Ts...,NaN,ONA,1,Other,Finished Airing,NaN,"Apr 24, 2019",...,NaN,NaN,NaN,[],[],[],[],[],[],[]
17053,17054.0,"Taka no Tsume-dan, Toyama",https://myanimelist.net/anime/33922/Taka_no_Ts...,NaN,ONA,Unknown,Original,Currently Airing,NaN,"Apr 6, 2016 to ?",...,NaN,NaN,NaN,[],[],[],[],[],[],[]


In [ ]:
final_anime_df[final_anime_df["Title"] == 'Akina to Onsen de H Shiyo!'	]

,Rank,Title,Link,Score,Type,# Episodes,Source,Status,Premiered,Aired Date,...,Adaptation,Sequel,Prequel,Characters,Role,Voice Actors,Openings,Opening Artists,Endings,Ending Artists
23049,NaN,Akina to Onsen de H Shiyo!,https://myanimelist.net/anime/9849/Akina_to_On...,6.41,OVA,1,Original,Finished Airing,NaN,"Nov 17, 2010",...,NaN,NaN,NaN,"['Miyazawa, Akina']",['Main'],"['Tanaka, Ryouko']",[],[],[],[]
23057,NaN,Akina to Onsen de H Shiyo!,https://myanimelist.net/anime/9849/Akina_to_On...,6.41,OVA,1,Original,Finished Airing,NaN,"Nov 17, 2010",...,NaN,NaN,NaN,"['Miyazawa, Akina']",['Main'],"['Tanaka, Ryouko']",[],[],[],[]


In [ ]:
path = '/content/drive/My Drive/24000_anime_stats_022123.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  final_anime_df.to_csv(f)


In [ ]:
final_anime_df[final_anime_df["Genres"] == 'Hentai,'].sort_values(by = "Score")

,Rank,Title,Link,Score,Type,# Episodes,Source,Status,Premiered,Aired Date,...,Adaptation,Sequel,Prequel,Characters,Role,Voice Actors,Openings,Opening Artists,Endings,Ending Artists
23999,NaN,Wake Up! Aria: Majokko Virgin Kiki Ippatsu,https://myanimelist.net/anime/5160/Wake_Up_Ari...,4.54,OVA,1,Unknown,Finished Airing,NaN,"Feb 20, 1998",...,NaN,NaN,NaN,[],['Background Art'],[],[],[],[],[]
23998,NaN,Dream Hazard: Akuma no Program,https://myanimelist.net/anime/4178/Dream_Hazar...,4.59,OVA,1,Manga,Finished Airing,NaN,"Feb 25, 1998",...,NaN,NaN,NaN,['Kaori'],['Main'],[],[],[],[],[]
23997,NaN,Sei Michaela Gakuen Hyouryuuki II,https://myanimelist.net/anime/2240/Sei_Michael...,4.59,OVA,2,Unknown,Finished Airing,NaN,"Oct 21, 1994 to Nov 25, 1995",...,NaN,NaN,Sei Michaela Gakuen Hyouryuuki,"['Kozue', 'Miuchi, Youko', 'Kumi', 'Gonza', 'S...","['Main', 'Supporting', 'Supporting', 'Supporti...",[],[],[],[],[]
23995,NaN,Aoi Taiken,https://myanimelist.net/anime/8574/Aoi_Taiken,4.61,OVA,2,Unknown,Finished Airing,NaN,"Oct 11, 1984 to Oct 7, 1985",...,NaN,NaN,NaN,[],[],[],[],[],[],[]
23993,NaN,Shirouto Club: Hamedori Anime Kanzen Mushuusei!!,https://myanimelist.net/anime/5551/Shirouto_Cl...,4.62,OVA,1,Unknown,Finished Airing,NaN,"Aug 20, 2003",...,NaN,NaN,NaN,[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22536,NaN,Ane wa Yanmama Junyuu-chuu,https://myanimelist.net/anime/41261/Ane_wa_Yan...,7.74,OVA,2,Manga,Finished Airing,NaN,"Jun 5, 2020 to Jul 3, 2020",...,NaN,NaN,NaN,"['Aika', 'Takuya']","['Main', 'Main']","['Mizuguchi, Matsuri']",[],[],[],[]
22535,NaN,Garden: Takamine-ke no Nirinka - The Animation,https://myanimelist.net/anime/50488/Garden__Ta...,7.76,OVA,Unknown,Manga,Currently Airing,NaN,"Feb 25, 2022 to ?",...,Garden,NaN,NaN,"['Takamine, Ayame', 'Takamine, Sayuri', 'Takam...","['Main', 'Main', 'Main']",[],[],[],[],[]
22534,NaN,Imaizumin Chi wa Douyara Gal no Tamariba ni Na...,https://myanimelist.net/anime/48755/Imaizumin_...,7.80,OVA,2,Manga,Finished Airing,NaN,"Aug 6, 2021",...,Imaizumin Chi wa Douyara Gal no Tamariba ni Na...,NaN,NaN,"['Hamazaki, Reina', 'Sasaki, Yukina', 'Kurumiz...","['Main', 'Main', 'Main', 'Main']","['Mizuguchi, Matsuri', 'Isshiki, Mayu', 'Osamu...",[],[],[],[]
22533,NaN,Master Piece The Animation,https://myanimelist.net/anime/39392/Master_Pie...,7.81,OVA,2,Manga,Finished Airing,NaN,"May 31, 2019 to Aug 30, 2019",...,Master_Piece,3Piece The Animation,NaN,"['Tsubakihara, Mira', 'Tsubakihara, Nina', 'Ts...","['Main', 'Main', 'Main', 'Main']","['Mizusawa, Kei', 'Sakurai, Harumi', 'Inokuchi...",[],[],[],[]
